In [72]:
import requests

req = requests.get(f'https://api.tzkt.io/v1/accounts/tz1RnsQi6BaYpm6sMMTkVqGk5Jk6BKpmTp8C/metadata')
req.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [67]:
req.status_code

204

In [30]:
import sys
import time
import datetime
from pathlib import Path
from tqdm import tqdm
import requests
from collections import Counter

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import lib.utils
import lib.iter_tr


stamp_step = 100000
config_hash = '4ff66f8cc8'
transactions_dir = repo_dir / 'cache' / 'transactions' / config_hash
assert transactions_dir.is_dir()

name2addr = {
    'art_house_contract': 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9',
    'curate_contract': 'KT1TybhR7XraG75JFYKSrh7KnxukMBT5dor6',
    'hdao_contract': 'KT1AFA2mwNUMNd4SsujE1YYp29vd8BZejyKW',
    'nft_contract': 'KT1RJ6PbjHpwc3M5rw5s2Nbmefwbuwbdxton',
    'comission_wallet': 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw',
}
addr2name = { addr: name for name, addr in name2addr.items() }

In [7]:
money_trs = []

for tr in lib.iter_tr.iter_tr(transactions_dir, stamp_step):
    tr_addrs = set()
    tr_volume = 0
    for op in tr['ops']:
        tr_volume += op['volume']
        tr_addrs.add(op['sender'])
        if 'receiver' in op:
            tr_addrs.add(op['receiver'])
    if not (tr_addrs & known_addrs) or tr_volume == 0:
        continue
    money_trs.append(tr)

len(money_trs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:04<00:00,  4.57it/s]

10630

In [80]:
def get_tr_money_delta(tr):
    res = {}
    for op in tr['ops']:
        if op['volume'] == 0:
            continue
        sender, receiver = op['sender'], op['receiver']
        res[sender] = res.get(sender, 0) - op['volume']
        res[receiver] = res.get(receiver, 0) + op['volume']
    return res


def get_addr_signature(addr):
    return addr2name.get(addr, '*')


def get_op_signature(op):
    if op['status'] != 'applied':
        return op['status']
    if op['type'] != 'transaction':
        return op['type']
    call = op.get('parameters', {'call': ''})['call']
    if op['volume'] > 0:
        return (
            call,
            get_addr_signature(op['sender']) + ' > ' + get_addr_signature(op['receiver']),
        )
    return call


def get_tr_signature(tr):
    ext_count = 0
    res = tuple()
    has_any_applied = False
    for op in tr['ops']:
        if op['type'] == 'reveal':
            continue
        if op['status'] == 'applied':
            has_any_applied = True
        if op['sender'] not in addr2name and op.get('receiver') not in addr2name:
            ext_count += 1
        else:
            res += (get_op_signature(op),)
    if not has_any_applied:
        return tr['ops'][0]['status']
    res = (ext_count > 0,) + res
    return res


from collections import Counter

tr_signatures = Counter()

for tr in money_trs:
    tr_signature = get_tr_signature(tr)
    if tr_signature == (False, ('', 'comission_wallet > *')):
        # assert len(tr['ops']) == 1
        # print(tr['hash'])
        # print(tr['time'], tr['ops'][0]['receiver'], tr['ops'][0]['volume'])
        pass
    if tr_signature == (False,
        ('collect', '* > art_house_contract'),
        ('', 'art_house_contract > *'),
        ('', 'art_house_contract > comission_wallet'),
        ('', 'art_house_contract > *'),
        'hDAO_batch',
        'transfer'):
            break
    tr_signatures[tr_signature] += 1

tr_signatures.most_common()

[((True, ('', '* > comission_wallet')), 2),
 ((False,
   ('collect', 'comission_wallet > art_house_contract'),
   ('', 'art_house_contract > *'),
   ('', 'art_house_contract > comission_wallet'),
   ('', 'art_house_contract > *'),
   'hDAO_batch',
   'transfer'),
  1)]

In [81]:
print(get_tr_money_delta(tr))

{'tz1Rux3pGrp2KpZUBKvg3VQ4edRYVJsapr4z': -10, 'KT1Hkg5qeNhfwpKW4fXvq7HGZB9z2EnmCCA9': 0.0, 'tz1ZdMfzmWLb8mu22jE7rZ8Y3t9iKezM68cq': 9.75, 'tz1UBZUkXpKGhYsP5KtzDNqLLchwF4uHrGjw': 0.25}
